# Modeling with GNN-ePC-SAFT

Model combining graph neural network with ePC-SAFT


## Starting point

In [1]:
%pip install torchmetrics clu ml-collections torch_geometric polars rdkit -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradient 2.0.6 requires attrs<=19, but you have attrs 23.1.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -U "ray[tune]" -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.23.4 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.23.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

In [ ]:
%pip show jax | grep Version

In [3]:
!nvidia-smi

Wed Jul 26 16:50:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A4000    Off  | 00000000:00:05.0 Off |                  Off |
| 41%   35C    P8     8W / 140W |      1MiB / 16376MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## PNA EPCSAFT

In [5]:
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
%cd GNN-ePC-SAFT

In [ ]:
!python train.py --workdir=./ --config=configs/default.py \
--config.num_train_steps={int(500000)} --config.log_every_steps=100 --config.num_para=7 \
--config.checkpoint_every_steps=2000 --config.learning_rate=0.0001 --config.patience=1000 \
--config.warmup_steps=100 --config.optimizer={'adam'} --config.batch_size=128 \
--config.propagation_depth=4 --config.hidden_dim=64 --config.num_mlp_layers=2 --config.pre_layers=1 --config.post_layers=3

In [ ]:
!python tuner.py --workdir=/notebooks/GNN-ePC-SAFT/ --config=configs/default.py \
--config.num_train_steps=10000 --config.log_every_steps=100\
--config.learning_rate=0.001  --config.optimizer={'adam'} --config.batch_size=128\
--config.num_para=7

In [3]:
!python tuner_restore.py --workdir=/notebooks/GNN-ePC-SAFT/ --config=configs/default.py \
--restoredir=./ray/tune_with_parameters_2023-07-25_14-28-40 \
--config.num_train_steps=10000 --config.log_every_steps=100\
--config.learning_rate=0.001 --config.optimizer={'adam'} --config.batch_size=128\
--config.num_para=7

x64 on
2023-07-27 01:15:16,860	ERROR utils.py:540 -- Unexpected error calculating docker cpuset ids.
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/ray/_private/utils.py", line 537, in _get_docker_cpus
    cpu_ids.append(int(num_or_range))
ValueError: invalid literal for int() with base 10: '\n'
2023-07-27 01:15:16,919	INFO worker.py:1621 -- Started a local Ray instance.
2023-07-27 01:15:17,959	INFO experiment_analysis.py:972 -- No trial data passed in during `ExperimentAnalysis` initialization -- you are most likely loading the experiment after it has completed.
Loading trial data from the experiment checkpoint file. This may result in loading some stale information, since checkpointing is periodic.
2023-07-27 01:15:18,573	INFO tune.py:666 -- [output] This will use the new output engine with verbosity 0. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see http

In [12]:
!python evaluate.py --workdir=./ --config=configs/default.py \
--config.num_train_steps={int(500000)} --config.log_every_steps=100 --config.num_para=7 \
--config.checkpoint_every_steps=2000 --config.learning_rate=0.0001 --config.patience=1000 \
--config.warmup_steps=100 --config.optimizer={'adam'} --config.batch_size=128 \
--config.propagation_depth=4 --config.hidden_dim=64 --config.num_mlp_layers=2 --config.pre_layers=1 --config.post_layers=3

x64 on
I0727 02:15:21.272435 140449394669376 evaluate.py:161] JAX host: 0 / 1
I0727 02:15:21.272600 140449394669376 evaluate.py:162] JAX local devices: [StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)]
I0727 02:15:21.272883 140449394669376 evaluate.py:164] Calling evaluate
wandb: Currently logged in as: wildsonbbl (wildson). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.13.4
wandb: Run data is saved locally in /notebooks/GNN-ePC-SAFT/wandb/run-20230727_021521-2l1ta29t
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run leafy-sea-545
wandb: ⭐️ View project at https://wandb.ai/wildson/gnn-pc-saft
wandb: 🚀 View run at https://wandb.ai/wildson/gnn-pc-saft/runs/2l1ta29t
I0727 02:15:22.057686 140449394669376 evaluate.py:69] Obtaining datasets.
inchis saved: 1940
I0727 02:15:23.364826 140449394669376 evaluate.py:90] Initi